In [1]:
try:
  import google.colab
  IN_COLAB = True
except:
  IN_COLAB = False


In [2]:
if IN_COLAB:
    !git clone https://github.com/ElsayedIbrahim98/mai_project1_optimization

fatal: destination path 'mai_project1_optimization' already exists and is not an empty directory.


In [3]:
if IN_COLAB:
    !pip3 install -r mai_project1_optimization/requirements.txt

In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import models
from torchvision.models import *
from plotly import express as px
from collections import Counter
import numpy as np
import random
import time
import psutil
import GPUtil
import threading
import time
import csv
from pynvml import *
from google.colab import files
import os
from datetime import datetime

if(IN_COLAB):
    from mai_project1_optimization.modules.dataset import IntelImageClassificationDataset
    from mai_project1_optimization.modules.utility import NotebookPlotter, InferenceSession, Evaluator, ISO_time
    from mai_project1_optimization.modules.trainer import Trainer, MetricsTrainer
else:
    from modules.dataset import IntelImageClassificationDataset
    from modules.utility import NotebookPlotter, InferenceSession, Evaluator, ISO_time
    from modules.trainer import Trainer

torch.manual_seed(1)
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(DEVICE)

def set_seed(seed=1):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True  # for reproducibility
    torch.backends.cudnn.benchmark = False

# Removed support for Tensor Units
# torch.backends.cudnn.allow_tf32 = True
# torch.backends.cuda.matmul.allow_tf32 = True

set_seed(1)

cuda


In [5]:
def start_gpu_logger(filename="gpu_usage_log.csv", interval=1, stop_event=None):
    nvmlInit()
    handle = nvmlDeviceGetHandleByIndex(0)

    def log():
        with open(filename, mode="w", newline="") as file:
            writer = csv.writer(file)
            writer.writerow(["Time", "GPU Utilization (%)", "Memory Used (MB)"])
            while not stop_event.is_set():
                util = nvmlDeviceGetUtilizationRates(handle)
                mem = nvmlDeviceGetMemoryInfo(handle)
                writer.writerow([
                    datetime.now().strftime("%H:%M:%S"),
                    util.gpu,
                    mem.used // 1024**2
                ])
                time.sleep(interval)
        nvmlShutdown()

    thread = threading.Thread(target=log)
    thread.start()
    return thread



In [6]:
choice = 1 # 1,2,3
freezeLayer = False

if choice != 5:
    dataset = IntelImageClassificationDataset(resize=(150,150))
else:
    dataset = IntelImageClassificationDataset(resize=(384,384))

# SqueezeNet 1.1
if choice == 1:
    model = models.squeezenet1_1(weights=SqueezeNet1_1_Weights.DEFAULT)
    num_features = model.classifier[1].in_channels
    kernel_size = model.classifier[1].kernel_size
    if(freezeLayer):
        for param in model.parameters():
            param.requires_grad = False
    model.classifier[1] = nn.Conv2d(num_features, 6, kernel_size)




dataloader = DataLoader(dataset.train_dataset, batch_size=128, shuffle=True, pin_memory=True)
#, pin_memory=True if torch.cuda.is_available() else False
#num_workers=2, prefetch_factor=2
trainer = Trainer(model=model, lr=0.001)

Downloading: "https://download.pytorch.org/models/squeezenet1_1-b8a52dc0.pth" to /root/.cache/torch/hub/checkpoints/squeezenet1_1-b8a52dc0.pth
100%|██████████| 4.73M/4.73M [00:00<00:00, 29.2MB/s]


In [7]:
# Step 1: Create the stop event
stop_event = threading.Event()

# Step 2: Start GPU logging
gpu_log_thread = start_gpu_logger("gpu_usage_log.csv", interval=1, stop_event=stop_event)

start_train = time.time()
trainer.train(dataloader=dataloader, epochs=10)
end_train = time.time()
training_time = end_train - start_train

stop_event.set()
gpu_log_thread.join()  # Ensure logging is done
files.download("gpu_usage_log.csv")


  0%|          | 0/10 [00:00<?, ?it/s]

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [8]:
session = InferenceSession(model)

# Inference output (already done in your notebook)
start_infer = time.time()
preds = session(torch.stack([item[0] for item in dataset.test_dataset]))
end_infer = time.time()
inference_time = end_infer - start_infer
true_labels = torch.tensor([item[1] for item in dataset.test_dataset])

# Classification Metrics
accuracy = Evaluator.acc(preds, true_labels).item()
precision = Evaluator.ppv(preds, true_labels).item()
recall = Evaluator.tpr(preds, true_labels).item()
f1 = Evaluator.f1_score(preds, true_labels).item()


In [9]:
def get_memory_usage():
    import psutil, os
    process = psutil.Process(os.getpid())
    return process.memory_info().rss / 1e6  # in MB

def get_gpu_utilization():
    try:
        gpus = GPUtil.getGPUs()
        return gpus[0].load * 100  # First GPU
    except:
        return None

def get_cpu_usage():
    return psutil.cpu_percent(interval=1)

training_memory = get_memory_usage()
gpu_utilization = get_gpu_utilization()
cpu_usage = get_cpu_usage()


In [10]:
import pandas as pd
from pathlib import Path

results = {
    "Experiment": "pin_memory + num_workers",  # customize for each run
    "Training Time (s)": round(training_time, 2),
    "Inference Time (s)": round(inference_time, 2),
    "Memory Usage (MB)": round(training_memory, 2),
    #"GPU Utilization": round(gpu_utilization, 2),
    #"CPU Usage": round(cpu_usage, 2),
    "Accuracy": round(accuracy, 4),
    #"Precision": round(precision, 4),
    #"Recall": round(recall, 4),
    #"F1 Score": round(f1, 4),

}

# Convert to DataFrame
results_df = pd.DataFrame([results])

# Append to CSV (create if it doesn't exist)
csv_path = "/content/optimization_report.csv"
results_df.to_csv(csv_path, mode='a', header=not Path(csv_path).exists(), index=False)

print(f"✅ Results saved to: {csv_path}")

✅ Results saved to: /content/optimization_report.csv
